LSTM Neuro Net for Anomally Detection

In [1]:
# import all required packages
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random
import pandas as pd
import torchtext.legacy #used for NLP

torch.backends.cudnn.deterministic = True

In [2]:
# Set parameters for the LSTM

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 22000
LEARNING_RATE = 1e-3
BATCH_SIZE = 128
NUM_EPOCHS = 25
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 220
HIDDEN_DIM = 440
OUTPUT_DIM = 1

## Dataset

In [3]:
# Review the data set before running the neuro net
# This cell is not used when building or running the neuro net
df = pd.read_csv('HDFS_2k-parsed_3-compl.csv', delimiter = "\t")
df.head()

,"review,sentiment"
0,"2,1"
1,"4,1"
2,"8,1"
3,"0,1"
4,"6,1"


In [4]:
# Delete info from previous cell
del df

In [5]:
# Define the label and text parameters

TEXT = torchtext.legacy.data.Field(sequential=True,
                  tokenize='spacy',
                  include_lengths=True) # necessary for packed_padded_sequence

LABEL = torchtext.legacy.data.LabelField(dtype=torch.float)

C:\Users\Graha\miniconda3\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [6]:
# Assign fields ('headers of csv')
# Get data from csv
fields = [('review', TEXT), ('sentiment', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
    path="HDFS_2k-parsed_3-compl.csv", format='csv',
    skip_header=True, fields=fields)

In [7]:
# Split data into training, validation and tests sets

train_data, valid_data, test_data = dataset.split(
    split_ratio=[0.75, 0.05, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 1495
Num Valid: 398
Num Test: 100


In [8]:
# Build vocabulary
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 13
Number of classes: 2


In [9]:
# Display count for normal and anomaly logs
LABEL.vocab.freqs

Counter({'1': 1444, '0': 51})

Make dataset iterators:

In [10]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True, # necessary for packed_padded_sequence
    sort_key=lambda x: len(x.review),
    device=DEVICE)

In [11]:
# Test the iterators
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break

Train
Text matrix size: torch.Size([1, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([1, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([2, 100])
Target vector size: torch.Size([100])


In [12]:
# Neuro net model
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_length):

        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        
        #[sentence len, batch size, embedding size] => 
        #  output: [sentence len, batch size, hidden size]
        #  hidden: [1, batch size, hidden size]
        packed_output, (hidden, cell) = self.rnn(packed)
        
        return self.fc(hidden.squeeze(0)).view(-1) # return 0 or 1

In [13]:
INPUT_DIM = len(TEXT.vocab)

torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Training

In [14]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.review
            logits = model(text, text_lengths.cpu())
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.sentiment.size(0)
            correct_pred += (predicted_labels.long() == batch_data.sentiment.long()).sum()
        return correct_pred.float()/num_examples * 100

In [15]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text, text_lengths = batch_data.review
        
        ### FORWARD AND BACK PROP
        logits = model(text, text_lengths.cpu())
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.sentiment)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/025 | Batch 000/012 | Cost: 0.7121
training accuracy: 96.59%
valid accuracy: 97.49%
Time elapsed: 0.01 min
Epoch: 002/025 | Batch 000/012 | Cost: 0.1340
training accuracy: 96.59%
valid accuracy: 97.49%
Time elapsed: 0.01 min
Epoch: 003/025 | Batch 000/012 | Cost: 0.0605
training accuracy: 96.59%
valid accuracy: 97.49%
Time elapsed: 0.02 min
Epoch: 004/025 | Batch 000/012 | Cost: 0.2062
training accuracy: 96.59%
valid accuracy: 97.49%
Time elapsed: 0.03 min
Epoch: 005/025 | Batch 000/012 | Cost: 0.1113
training accuracy: 96.59%
valid accuracy: 97.49%
Time elapsed: 0.04 min
Epoch: 006/025 | Batch 000/012 | Cost: 0.0866
training accuracy: 96.59%
valid accuracy: 97.49%
Time elapsed: 0.04 min
Epoch: 007/025 | Batch 000/012 | Cost: 0.1708
training accuracy: 96.59%
valid accuracy: 97.49%
Time elapsed: 0.05 min
Epoch: 008/025 | Batch 000/012 | Cost: 0.1853
training accuracy: 96.66%
valid accuracy: 97.49%
Time elapsed: 0.06 min
Epoch: 009/025 | Batch 000/012 | Cost: 0.1069
training a